In [1]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader,random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [2]:
IMAGE_X = 50
IMAGE_Y = 50

TRAIN_DATA_PATH = 'data/train/'

VALIDATION_SPLIT_SIZE = 0.1

BATCH_SIZE = 512
NUM_WORKERS = 8
SHUFFLE = True

LR = 0.01
EPOCH = 10

TRANSFORM = transforms.Compose(
[transforms.Resize((IMAGE_X,IMAGE_Y)),
 transforms.ToTensor(),
 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:


def load_dataset_from_folder():
    all_data = ImageFolder(
        root = TRAIN_DATA_PATH,
        transform = TRANSFORM
    )
    
    validation_size = int(VALIDATION_SPLIT_SIZE * len(all_data))
    train_size = len(all_data) - validation_size
    train_dataset, test_dataset = torch.utils.data.random_split(all_data, [train_size, validation_size])

    training_data_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=SHUFFLE
    )
    
    validation_dataset_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=SHUFFLE
    )
        
    return training_data_loader,validation_dataset_loader

class Net(nn.Module):
    fin_x = 0
    fin_y = 0
    k_size = 3
    stride = 2
    pool_size = 2
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, self.k_size, self.stride),
            nn.Dropout2d(),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size, self.pool_size))

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, self.k_size, self.stride),
            nn.Dropout2d(),
            nn.ReLU(),
            nn.MaxPool2d(self.pool_size, self.pool_size))
        
        self.drop_out = nn.Dropout()

        
        tmpx = int((int((IMAGE_X-self.k_size)/self.stride)+1)/self.pool_size)
        tmpy = int((int((IMAGE_Y-self.k_size)/self.stride)+1)/self.pool_size)
        
        self.fin_x = int((int((tmpx-self.k_size)/self.stride)+1)/self.pool_size)
        self.fin_y = int((int((tmpy-self.k_size)/self.stride)+1)/self.pool_size)
        
        self.fc1 = nn.Sequential(
            nn.Linear(64 * self.fin_x * self.fin_y, 2048),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(2048, 4),
            nn.ReLU()
        )
        
        self.sof = nn.LogSoftmax()

    def forward(self, inp):
        out = self.layer1(inp)
        out = self.layer2(out)
        out = out.view(-1, 64 * self.fin_x * self.fin_y)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sof(out)
        return out

In [4]:
model = Net()

# Defining the optimizer
optimizer = Adam(model.parameters(), lr=LR)

# Defining the loss function
criterion = nn.CrossEntropyLoss()

# Checking if GPU is available
if torch.cuda.is_available():
    print('Found Cuda')
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): Dropout2d(p=0.5, inplace=False)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Sequential(
    (0): Linear(in_features=256, out_features=2048, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=2048, out_features=4, bias=True)
    (1): ReLU()
  )
  (sof): LogSoftmax()
)


In [5]:
train_generator,test_generator = load_dataset_from_folder()

In [6]:
for epoch in range(EPOCH):

    running_loss = 0.0
    for step,(x,y) in enumerate(train_generator):
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('[%d] Training loss: %.3f' %(epoch + 1, running_loss))

    validation_loss = 0.0
    with torch.no_grad():
        for step,(x,y) in enumerate(test_generator):
            outputs = model(x)
            loss = criterion(outputs, y)
            validation_loss += loss.item()
    print('[%d] Validation loss: %.3f' %(epoch + 1, validation_loss))

[1] Training loss: 3.321


/home/ankurshaswat/MyFiles/Repos/COL780/env/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1] Validation loss: 1.413
[2] Training loss: 2.903
[2] Validation loss: 1.382
[3] Training loss: 2.774
[3] Validation loss: 1.386
[4] Training loss: 2.771
[4] Validation loss: 1.373


KeyboardInterrupt: 